# Exploring Seasonal NDVI pattern over Punjab, India for 2023



In [6]:
#import libraries

import os
import pathlib
import os

import geopandas as gpd
import pandas as pd
import panel as pn
import rioxarray as rxr
import xarray as xr
import holoviews as hv
from pathlib import Path


import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt


In [12]:
from pathlib import Path
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# paths
base = Path("data")

boundary_path = base / "boundaries" / "gadm41_IND_1.shp"
ndvi_dir = base / "ndvi"

In [9]:
adm1 = gpd.read_file(boundary_path)

punjab = adm1[adm1["NAME_1"].str.contains("Punjab", case=False)].copy()
punjab = punjab.to_crs(4326)

punjab


,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
30,IND.28_1,IND,India,Punjab,NA,NA,State,State,NA,IN.PB,IN-PB,"POLYGON ((75.90314 29.74628, 75.89723 29.75553..."


In [10]:
tif_dir = ndvi_dir / "tif"
tif_dir.mkdir(exist_ok=True)

hdf_files = sorted(ndvi_dir.glob("*.hdf"))
hdf_files[:3], len(hdf_files)


([PosixPath('data/ndvi/MOD13Q1.A2023033.h23v05.061.2023054044314.hdf'),
  PosixPath('data/ndvi/MOD13Q1.A2023033.h24v05.061.2023054043151.hdf'),
  PosixPath('data/ndvi/MOD13Q1.A2023033.h24v06.061.2023054042946.hdf')],
 88)

In [13]:
for hdf in hdf_files:
    print("Processing:", hdf.name)

    ndvi_sub = f'HDF4_EOS:EOS_GRID:"{hdf}":MOD_Grid_16DAY_250m_500m_VI:250m 16 days NDVI'

    with rasterio.open(ndvi_sub) as src:
        arr = src.read(1).astype(float)
        arr[arr == src.nodata] = np.nan
        arr = arr * 0.0001    # apply scale factor

        profile = src.profile
        profile.update(driver="GTiff", dtype="float32", count=1, nodata=np.nan)

        out = tif_dir / (hdf.stem + "_NDVI.tif")
        with rasterio.open(out, "w", **profile) as dst:
            dst.write(arr, 1)

print("Done!")


Processing: MOD13Q1.A2023033.h23v05.061.2023054044314.hdf


RasterioIOError: HDF4_EOS:EOS_GRID:"data/ndvi/MOD13Q1.A2023033.h23v05.061.2023054044314.hdf":MOD_Grid_16DAY_250m_500m_VI:250m 16 days NDVI: No such file or directory